In [9]:
import ast
import pandas as pd
import numpy as np

In [10]:
prediction_df = pd.read_csv('test_prediction003.csv')
prediction_df['ground_truth'] = prediction_df.apply(lambda row: ast.literal_eval(row.ground_truth), axis=1)
prediction_df['boxes'] = prediction_df.apply(lambda row: ast.literal_eval(row.boxes), axis=1)

In [11]:
prediction_df

,filename,width,height,class,xmin,ymin,xmax,ymax,image_id,labels,boxes,scores,ground_truth
0,2-5398.jpg,1280,720,starfish,179,156,228,203,0,"['starfish', 'starfish', 'starfish', 'starfish...","[[761, 59, 794, 96], [886, 136, 927, 172], [26...","[0.6104638, 0.60373336, 0.25634515, 0.13928367...","[179, 156, 228, 203]"
1,1-9381.jpg,1280,720,starfish,438,681,521,720,1,"['starfish', 'starfish', 'starfish', 'starfish...","[[1209, 213, 1270, 270], [526, 400, 560, 447],...","[0.2355324, 0.23038493, 0.11747926, 0.1048326,...","[438, 681, 521, 720]"
2,2-5456.jpg,1280,720,starfish,4,643,63,698,2,"['starfish', 'starfish', 'starfish', 'starfish...","[[334, 214, 379, 246], [443, 196, 471, 228], [...","[0.29690042, 0.1926385, 0.07696684, 0.06669554...","[4, 643, 63, 698]"
3,2-5843.jpg,1280,720,starfish,1080,620,1164,678,3,"['starfish', 'starfish', 'starfish', 'starfish...","[[728, 319, 766, 357], [813, 635, 850, 664], [...","[0.87733483, 0.43130508, 0.40441397, 0.2223332...","[1080, 620, 1164, 678]"
4,2-6368.jpg,1280,720,starfish,169,145,202,173,4,"['starfish', 'starfish', 'starfish', 'starfish...","[[1177, 558, 1226, 628], [470, 345, 503, 384],...","[0.1899294, 0.09701449, 0.080434814, 0.0619859...","[169, 145, 202, 173]"
5,1-9243.jpg,1280,720,starfish,105,654,150,699,5,"['starfish', 'starfish', 'starfish', 'starfish...","[[454, 587, 491, 612], [434, 150, 470, 179], [...","[0.22215822, 0.19168675, 0.14077759, 0.1266641...","[105, 654, 150, 699]"
6,2-7967.jpg,1280,720,starfish,514,299,549,319,6,"['starfish', 'starfish', 'starfish', 'starfish...","[[1224, 1, 1259, 21], [995, 26, 1043, 55], [71...","[0.38565028, 0.24674265, 0.19437331, 0.1698201...","[514, 299, 549, 319]"
7,2-5439.jpg,1280,720,starfish,186,352,235,399,7,"['starfish', 'starfish', 'starfish', 'starfish...","[[974, 534, 1013, 565], [224, 147, 257, 182], ...","[0.4448414, 0.24991387, 0.19835176, 0.13883433...","[186, 352, 235, 399]"
8,2-8865.jpg,1280,720,starfish,448,597,508,636,8,"['starfish', 'starfish', 'starfish', 'starfish...","[[459, 594, 499, 629], [0, 0, 55, 36], [360, 0...","[0.15105337, 0.09009042, 0.07866978, 0.0663505...","[448, 597, 508, 636]"
9,2-5508.jpg,1280,720,starfish,246,607,287,646,9,"['starfish', 'starfish', 'starfish']","[[498, 554, 528, 585], [154, 124, 206, 175], [...","[0.115978815, 0.07435075, 0.07144379]","[246, 607, 287, 646]"


In [13]:
beta = 2
fBeta_list = []
for IoU_threshold in np.arange(0.3,0.85,0.05):
  true_positive = 0
  false_positive = 0
  false_negative = 0
  IoU = 0
  for n in range(len(prediction_df)):
    sample_GT = prediction_df.at[n,'ground_truth']
    sample_img_GT = np.zeros((720,1280), dtype=float)
    sample_img_GT[sample_GT[1]:sample_GT[3], sample_GT[0]:sample_GT[2]] = 1
    if len(prediction_df.at[n,'boxes']) == 0:
      false_negative += 1
      continue
    for m in range(len(prediction_df.at[n,'boxes'])):
      sample_pred = prediction_df.at[n,'boxes'][m]
      sample_img_pred = np.zeros((720,1280), dtype=float)
      sample_img_pred[sample_pred[1]:sample_pred[3], sample_pred[0]:sample_pred[2]] = 1
      intersection_img = np.multiply(sample_img_GT, sample_img_pred)
      union_img = np.add(sample_img_GT, sample_img_pred)
      IoU = np.count_nonzero(intersection_img > 0)/np.count_nonzero(union_img>0)
      if IoU >= IoU_threshold:
        true_positive += 1
      else:
        false_positive += 1
  fBetaScore = ((1+beta**2)*true_positive)/((1+beta**2)*true_positive + false_negative * beta**2 + false_positive)
  fBeta_list.append(fBetaScore)

In [14]:
average_F2_score = sum(fBeta_list)/len(fBeta_list)
average_F2_score

0.09666208328145047